# El clasificador de galletas navideñas
Es una aplicación de visión de computadora basada en la arquitectura de redes convulosionales de resnet. Con el gran poder de decidir, sin preferencias que galletas son bonitas o feas, algo muy importante en estas fechas. Solo agrega el archivo al botón y se clasificará inmediatamente con una confianza asociada, esta confianza corresponde a cuan seguro está el modelo de su decisión.

In [5]:
!pip install fastai
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
from fastai.vision.widgets import *
!pip install voila

ModuleNotFoundError: ignored

In [ ]:
!pip install voila

     |████████████████████████████████| 430kB 5.6MB/s 
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.1 which is incompatible.
  Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1


In [6]:
!pip install tornado==5.1.0

     |████████████████████████████████| 522kB 5.9MB/s 
  Created wheel for tornado: filename=tornado-5.1-cp36-cp36m-linux_x86_64.whl size=461865 sha256=3d751799bb81d10cd7001ea88ec60e32591691545254ae8f1558ca0bfd0d8ff5
  Stored in directory: /root/.cache/pip/wheels/a3/fc/8b/cc9cae4f0c829663a49cc652ce7d994fa65434d61bb819c3d5
Successfully built tornado
  Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1


In [4]:
from fastai import *
from fastbook import *
from fastai.vision.widgets import *

In [5]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
path=Path('/content/gdrive/My Drive//Colab Notebooks/Galletas')
#path.ls


Mounted at /content/gdrive


In [7]:
fns = get_image_files(path)
fns

(#52) [Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/ugly-cookies.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/icecookies6-5a0f86899e94270037d9b4c8-5b043d4f43a10300381334be.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/gingerbread-man-cookie-vertical-a-1800.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/homemade-gingerbread-stars.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/imahges.jpeg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/christmas-sugar-cookies-featured-1024x768.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/image1.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/exps21624_CC153599C01_09_3b-696x696.jpg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/imaggges.jpeg'),Path('/content/gdrive/My Drive/Colab Notebooks/Galletas/Feas/gg.jpeg')...]

In [8]:
galletas = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [9]:
dls = galletas.dataloaders(path)

In [10]:
galletas = galletas.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = galletas.dataloaders(path)

learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(6)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


epoch,train_loss,valid_loss,error_rate,time
0,nan,1.114389,0.500000,00:11


/usr/local/lib/python3.6/dist-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,error_rate,time
0,nan,1.114389,0.500000,00:02
1,nan,1.114389,0.500000,00:02
2,nan,1.114389,0.500000,00:02
3,nan,1.114389,0.500000,00:02
4,nan,1.114389,0.500000,00:02
5,nan,1.114389,0.500000,00:02


In [11]:
learn.export(os.path.abspath('./my_export.pkl'))

In [12]:
learn_inf = load_learner(os.path.abspath('./my_export.pkl'),cpu=True)

In [13]:
button=widgets.FileUpload()
out=widgets.Output()
prediction=widgets.Label()

In [14]:
def click(change):
  img=PILImage.create(button.data[-1])
  out.clear_output()
  with out:display(img.to_thumb(120,120))
  pred,index,proba=learn_inf.predict(img)
  prediction.value=f'Se define como una de las galletas {pred}, y esto con una confianza de {proba[index]}'

In [15]:
button.observe(click,names=['data'])
display(VBox([widgets.Label('Ingrese una Galleta'),button,out,prediction]))